In [1]:
import gmpy2 as gmp
import random
import base64
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

#---------------------------------------------------------------------------------------------------------------------------------------------------------
#   sgn0 "sign" of x: returns -1 if x is lexically larger than  -x and, else returns 1
#   https://datatracker.ietf.org/doc/html/draft-irtf-cfrg-hash-to-curve-05#section-4.1
#---------------------------------------------------------------------------------------------------------------------------------------------------------
def FpMsgn0(x,p):
    thresh = (p - 1) // 2
    sign = 0
    for xi in reversed(x._raw):
        if xi > thresh:
            sign = -1 if sign == 0 else sign
        elif xi > 0:
            sign = 1 if sign == 0 else sign
    sign = 1 if sign == 0 else sign
    return sign

def Fpsgn0(x,p):
    thresh = (p - 1) // 2
    sign = 0
    if x > thresh:sign = -1 if sign == 0 else sign
    elif x > 0:sign = 1 if sign == 0 else sign
    sign = 1 if sign == 0 else sign
    return sign
#---------------------------------------------------------------------------------------------------------------------------------------------------------
#   I2OSP converts a nonnegative integer to an octet string of a specified length. RFC 3447, section 4.1 https://datatracker.ietf.org/doc/html/rfc3447#section-4.1
#---------------------------------------------------------------------------------------------------------------------------------------------------------
def I2OSP(x,length):
    if x < 0 or x >= (1 << (length<<3)):return None
    else :
        res=[0]*length
        for i in reversed(range(length)):
            res[i]=x & 0xFF
            x=x>>8
        return bytes(res)
#---------------------------------------------------------------------------------------------------------------------------------------------------------
#   OS2IP converts an octet string to a nonnegative integer. RFC 3447, section 4.2 https://datatracker.ietf.org/doc/html/rfc3447#section-4.2
#---------------------------------------------------------------------------------------------------------------------------------------------------------
def OS2IP(byts):
    res = 0
    for i in byts:res = (res << 8)+i
    return res    

P256params = {
            "Field_Prime":gmp.mpz(0xffffffff00000001000000000000000000000000ffffffffffffffffffffffff),
            "Curve_A": gmp.mpz(0xffffffff00000001000000000000000000000000fffffffffffffffffffffffc),
            "Curve_B": gmp.mpz(0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b),
            "Curve_Order": gmp.mpz(0xffffffff00000000ffffffffffffffffbce6faada7179e84f3b9cac2fc632551),
            "Generator_X":gmp.mpz(0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296),
            "Generator_Y":gmp.mpz(0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5)
            }

    
def  EllipticCurve(params):
    _p  = params["Field_Prime"]
    _a  = params["Curve_A"]
    _b  = params["Curve_B"]   
    _n  = params["Curve_Order"]   
    _gx = params["Generator_X"]
    _gy = params["Generator_Y"]
    

    def _addAffine(x1, y1, x2, y2, p):                  # Two points addition on elliptic curve with respect to affine coordinates scheme
        lamda = ((y1 - y2) * gmp.invert(x1 - x2, p)) % p
        return (x:=((lamda**2) - x1 - x2) % p) , (lamda * (x1 - x) - y1) % p
    
    def _doubleAffine(x1, y1, p):                       # Point doubling on elliptic curve with respect to affine coordinates scheme
        lamda = (3 * (x1**2)+_a) * gmp.invert(2 * y1, p)
        return (x:=((lamda**2) - (2 * x1)) % p) , (lamda * (x1 - x) - y1) % p
    
    def _rsqrt(x,p):
        if p & 3==3:
            res = pow(x,(p+1)>>2,p) 
            if ((res**2) * pow(x, -1 , p )) % p != 1: return None ## check if 'x' is a quadratic Residu modulo p using Euler's Criterion
            else: return res

    class EllipticCurve :
        def __init__(self,x,y=None,dotest=False):           # Class's Constructeur
            if x is None:                                   # If x is None, it represents the point at infinity.             
                self.infinity = True
                self.x, self.y = 0, 0                       #(0, 0), representation of the point at infinity in ECC
            else:
                if y is None:                               #If y is not provided
                    self.x = x % _p if type(x)==gmp.mpz else gmp.mpz(x) % _p    # Compute y of the point based on x using the _rsqrt function.
                    _ = _rsqrt((x**3+x*_a+_b) % _p,_p)
                    if (_ != None) : self.infinity, self.y, self.z = False, _, gmp.mpz(1)
                    else :raise TypeError("Invalide curve point parametres ...")
                else:           # If y is provided, validates the curve parameters and initializes the point.
                                # If y is already an instance of gmp.mpz, then y % _p is calculated directly.
                                # else  converts y into a gmp.mpz object using gmp.mpz(y) and then performs the modulus operation with _p
                    self.x = x % _p if type(x)==gmp.mpz else gmp.mpz(x) % _p
                    self.y = y % _p if type(y)==gmp.mpz else gmp.mpz(y) % _p
                    self.z = gmp.mpz(1)
                    self.infinity = False
                    # checks(x, y) lies on the curve equation y^2 ≡ x^3 + ax + b (mod p). If not, it raises a TypeError
                    if (y**2 % _p != (x**3+x*_a+_b) % _p) : raise TypeError("Invalide curve point parametres ...")
       
        def __str__(self):      # provide string representations of the curve points.
            return "("+str(self.x)+" , "+str(self.y)+")" if not self.infinity else "Infinity"
        __repr__ = __str__  
          
        
        def  tobytes(self):
            #     Point compression/Serialization as described by ZCach serialization format
            #     https://www.ietf.org/archive/id/draft-irtf-cfrg-pairing-friendly-curves-11.html#name-zcash-serialization-format-
            C_bit, I_bit, S_bit = 1, int(self.infinity), 0 if self.infinity else ((1+Fpsgn0(self.y, _p)) >>1)
            m_byte   = (C_bit << 7) | (I_bit << 6) | (((S_bit + 1) >> 1) << 5)
            x_string = I2OSP(0, _sizeInBytes) if self.infinity else I2OSP(self.x, _sizeInBytes)
            return bytes([m_byte]) + x_string 
        
        def fromBytes(bytearray):
            #     Point de-compression/de-Serialization as described by ZCach serialization format
            #     https://www.ietf.org/archive/id/draft-irtf-cfrg-pairing-friendly-curves-11.html#name-zcash-serialization-format-
            m_byte= bytearray[0] & 0xE0
            bytearray = bytearray[1:]
            if (m_byte == 0xE0) : raise TypeError("Invalide compressed point format ...") 
            if m_byte & 0x80 != 0 :
                if len(bytearray) != _sizeInBytes : raise TypeError("Invalide compressed point format ...")
            else :
                if len(bytearray) != (_sizeInBytes << 1) : raise TypeError("Invalide compressed point format ...")
            if (m_byte & 0x40 != 0) :
                if (any([e != 0 for e in bytearray])) : raise TypeError("Invalide compression of an infinity point ...")
                else : return EllipticCurve(None)
            else :
                if len(bytearray) == (_sizeInBytes << 1) :
                    x = gmp.mpz(OS2IP(bytearray[:_sizeInBytes]))
                    y = gmp.mpz(OS2IP(bytearray[_sizeInBytes:]))
                    return EllipticCurve(x, y, dotest = True)
                else :
                    x = gmp.mpz(OS2IP(bytearray))
                    y = _rsqrt((x**3 +x*_a+_b) % _p, _p)
                    if y is None : raise TypeError("Invalide point: not in the curve ...")
                    else :
                        if ((Fpsgn0(y,_p) + 1)>>1) == int(m_byte & 0x20 != 0) : return EllipticCurve(x,y)
                        else : return EllipticCurve(x, _p-y)

        def toBase64(self):
         return base64.b64encode(self.tobytes()).decode("ascii")
    
        def fromBase64(str):
         return EllipticCurve.fromBytes(base64.b64decode(str)) 
        
        def __add__(p,q):
            if p.infinity:
                if q.infinity : return EllipticCurve(None)
                else : return EllipticCurve(q.x,q.y)
            else:
                if q.infinity : return p.copy()
                else:
                    if p.x == q.x:
                        if p.y == q.y:return EllipticCurve((res := _doubleAffine(p.x, p.y))[0], res[1])
                        else : return EllipticCurve(None)
                    else : 
                        res =_addAffine(p.x,p.y,q.x,q.y,_p) 
                        return EllipticCurve(res[0],res[1])

        def __neg__(p)  : return EllipticCurve(p.x,-p.y)
        def __sub__(p,q): return p + (-q)               # invoke the __add__ method
        def __eq__(p,q) : return (p.x == q.x) & (p.y == q.y) if not(p.infinity | q.infinity) else (p.infinity == q.infinity)
        def __ne__(p,q) : return (p.infinity != q.infinity) | (p.x != q.x) | (p.y != q.y)
       
        def __rmul__(p,b):
            if (type(b) != int) & (type(b) != gmp.mpz) : raise TypeError("Invalide scalar value for multiplication ...")
            else :           
                if b==2:
                    _res = _doubleAffine(p.x,p.y,_p)
                    return EllipticCurve(_res[0],_res[1])
                else :
                    b=abs(b)
                    mask = (1 << (b.bit_length() - 2))  
                    _res = EllipticCurve(p.x, p.y)
                    while (mask!=0):
                        _res = 2 * _res
                        if (mask & b) != 0: _res = _res + p
                        mask = mask >> 1
                    return _res
       
        def RandomPoint():
            _x = random.randint(0, _p - 1)              # generates a random integer _x as the x coordinate
            _y = _rsqrt(_x**3+_a * _x + _b, _p)         # Computes the y-coordinate 
            while (_y == None):                         # Repeat until a valid point is found
                _x = _x + 1                             # Increments x and revaluate y         
                _y = _rsqrt(_x**3+_a * _x + _b, _p)
            return EllipticCurve(_x,_y)                 # An instance of the EllipticCurve class is created with these coordinates and returned as the random point on the elliptic curve.    

        def RandomScalar(): return random.randint(0, _n - 1)
        def GetGenerator(): return EllipticCurve(_gx,_gy)
        def generer_k() : return get_random_bytes(32)
           
 
        def encrypt_aes_cbc(message, key):
            iv = get_random_bytes(16)  # Générer un IV aléatoire de 16 octets
            padding_size = 16 - (len(message) % 16)

            message_padded = message + (chr(padding_size) * padding_size).encode()  # Ajouter un padding

            cipher = AES.new(key, AES.MODE_CBC, iv)  # Créer l'objet de chiffrement
            ciphertext = cipher.encrypt(message_padded)  # Chiffrer le message

            return base64.b64encode(iv + ciphertext).decode()  # Retourner un message encodé en base64
        
        def decrypt_aes_cbc(encrypted_message, key):
           encrypted_data = base64.b64decode(encrypted_message)  # Décodage Base64
           iv = encrypted_data[:16]  # Extraire l'IV (16 premiers octets)
           ciphertext = encrypted_data[16:]  # Extraire le texte chiffré

           cipher = AES.new(key, AES.MODE_CBC, iv)  # Création du déchiffreur
           decrypted_padded = cipher.decrypt(ciphertext)  # Déchiffrer le message
           padding_size = decrypted_padded[-1]  # Lire le dernier octet (taille du padding)
           decrypted_message = decrypted_padded[:-padding_size]  # Supprimer le padding
           return decrypted_message.decode()  # Retourner le texte en clair


    
    EllipticCurve.Order = _n
    EllipticCurve.Prime = _p
    _sizeInBytes = (_p.bit_length() // 8)+int(_p.bit_length() % 8 != 0)
    return EllipticCurve    


In [3]:
curve  = EllipticCurve(P256params)
g = curve.GetGenerator()
n= curve.Order

#les clés Alex 
S = curve.RandomScalar()
Pk = S * g  #clé chiffrement
Pk = Pk.toBase64()

#les clés bob 

S_b = curve.RandomScalar()
Pk_b = S_b * g  #clé chiffrement
Pk_b_2 = Pk_b.toBase64()


print("-----------------------------------------------")
#les clés de alex
print("les clés de alex")
print(f"cle public est : {Pk}")
print(f"cle privée est : {S}")

#les clés de bob
print("les clés de bob")
print(f"cle public est : {Pk_b}")
print(f"cle privée est : {S_b}")

print("-----------------------------------------------")

# chifferement AES

k_aes = curve.generer_k() #clé AES 
k_int =  int.from_bytes(k_aes,byteorder='big') 

k2 = k_int * g  #clé chiffrement


k2_byte = k2.tobytes()[:32]


message = "Bonjour bob, voici un message secret !".encode()
c = curve.encrypt_aes_cbc(message,k2_byte)
print(f"message chiffer : {c}")



Pkb = curve.fromBase64(Pk_b_2)  

send= k_int * Pkb
sende = send .tobytes()


Sb_inverse = pow(S_b, -1, n)
new_k2 = send.__rmul__(Sb_inverse)
new_k2_byte = new_k2.tobytes()[:32]

m = curve.decrypt_aes_cbc(c,new_k2_byte)
print(f"message Déchiffré  : {m}")



-----------------------------------------------
les clés de alex
cle public est : oE0S77zpaQfJ6JFG5Sz7L6BbHGjiCOEIJ7DCRs0DkPBV
cle privée est : 601265529266839314881270249542084206582168582866974610670171111843278636483
les clés de bob
cle public est : (56631102306128862044787035536397795171707388100986574294739099982194354217662 , 101955544197760826937869903730647716850695431310575414745162677645622685943473)
cle privée est : 101670677182391508399240779466047394441275802381802330461334597751381194861100
-----------------------------------------------
message chiffer : i9nK4hPHPMUTknUYkcC/PI1+8yaLOLjM19IrXoim54aFRvzb6DOsUD2jtFWayufn1gsn3k5T8q+oXEmkqVeyUw==
message Déchiffré  : Bonjour bob, voici un message secret !
